<div class="alert alert-block alert-info">
<H1>
SpaceX  Falcon 9 Dashboard*
</H1></div>

Source for plot colors: https://stackoverflow.com/questions/70865163/set-background-color-in-dash-plotly
Source for app background colors: https://dash.plotly.com/layout
Source for color picker: https://www.rapidtables.com/web/color/color-picker.html   

### Import Required Libraries

In [1]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import pandas as pd # To read file
import plotly.express as px # To make figures

<ipython-input-1-91760cbcf454>:2: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-91760cbcf454>:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


<div class="alert alert-block alert-success">
<H5>
Read File
</H5></div>

In [2]:
csv_file = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_launch_data = pd.read_csv(csv_file)
print(spacex_launch_data.tail(3))

    Unnamed: 0  Flight Number   Launch Site  class  Payload Mass (kg)  \
53          53             52  CCAFS SLC-40      0             2647.0   
54          54             53  CCAFS SLC-40      1              362.0   
55          55             56  CCAFS SLC-40      0             5384.0   

   Booster Version Booster Version Category  
53  F9 B4  B1039.2                       B4  
54   F9 B4 B1045.1                       B4  
55  F9 B4  B1040.2                       B4  


**** Check Launch Sites as Unique List ****

In [3]:
df2 = pd.unique(spacex_launch_data[['Launch Site']].values.ravel())
print(df2)

['CCAFS LC-40' 'VAFB SLC-4E' 'KSC LC-39A' 'CCAFS SLC-40']


<div class="alert alert-block alert-success">
<H5>
Create App Filters for Filtering Data by Launch Site
</H5></div>

In [4]:
unique_launch_sites = spacex_launch_data['Launch Site'].unique().tolist()
launch_sites = []
launch_sites.append({'label': 'All Sites', 'value': 'All Sites'})
for launch_site in unique_launch_sites:
    launch_sites.append({'label': launch_site, 'value': launch_site})

<div class="alert alert-block alert-success">
<H5>
Calculate Min & Max Payloads
</H5></div>

In [5]:
#spacex_launch_data['Payload Mass (kg)'] = pd.to_numeric(spacex_launch_data['Payload Mass (kg)'])
spacex_launch_data['Payload Mass (kg)'] = spacex_launch_data['Payload Mass (kg)'].astype(int)

max_payload = spacex_launch_data['Payload Mass (kg)'].max()
min_payload = spacex_launch_data['Payload Mass (kg)'].min()

print(max_payload)
print(min_payload)

9600
0


<div class="alert alert-block alert-success">
<H5>
Create Slider Range Markers
</H5></div>

In [6]:
marks_dict = {}
for i in range(0,11000,1000):
    marks_dict[i] = {'label': str(i)+' Kg'}
print(marks_dict)

{0: {'label': '0 Kg'}, 1000: {'label': '1000 Kg'}, 2000: {'label': '2000 Kg'}, 3000: {'label': '3000 Kg'}, 4000: {'label': '4000 Kg'}, 5000: {'label': '5000 Kg'}, 6000: {'label': '6000 Kg'}, 7000: {'label': '7000 Kg'}, 8000: {'label': '8000 Kg'}, 9000: {'label': '9000 Kg'}, 10000: {'label': '10000 Kg'}}


<div class="alert alert-block alert-success">
<H5>
Create Background Color Palette for Figures
</H5></div>

In [7]:
# Bckground color for App Skeleton
colors = {
    'background': '#F6FFDB',
    'text': '#161A0B'
}

# Bckground color for Slider Bar
colors1a = {
    'background': '#751875',
    'text': '#FFFF4B'
}

# Bckground color for Slider Chart
colors1b = {
    'background': '#FFEFFF',
    'text': '#161A0B'
}

# Bckground color for Pie Chart
colors2 = {
    'background': '#F7FFE9',
    'text': '#161A0B'
}

<div class="alert alert-block alert-success">
<H5>
Create App Layout
</H5></div>

<div class="alert alert-block alert-warning">
<H5>
TASK 1: App Layout
</H5></div>

In [8]:
# Create a dash application
app = dash.Dash('SPACE X DASHBOARD')

# Create an app layout
app.layout = html.Div(style= {'backgroundColor': colors['background']}, 
                      children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(
                                    id = 'site-dropdown',
                                    style={'text-align-last': 'center', 'width':'80%', "color": "darkblue", 
                                           'padding':'3px', 'font-size': '20px'},
                                    options = launch_sites,
                                    placeholder = 'Select a Launch Site',
                                    searchable = True ,
                                    value = 'All Sites' # To set dropdown default to 'All Sites'
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                #########################
                               
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                html.Div([
                                    dcc.RangeSlider(
                                        id = 'payload_slider',
                                        min = 0,
                                        max = 10000,
                                        step = 1000,
                                        marks = marks_dict, # Created Above
                                        #marks= {marks_dict: {'style': {'color': 'red'}}},
                                        value = [min_payload, max_payload] # Calculated Above
                                    ),
                                ], style={'padding': '40px 30px'}),
                                
                                #marks={'1': '1', '6': '6', '12': {'label': '12', 'style': {'color': 'red'}}}

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

<div class="alert alert-block alert-warning">
<H5>
TASK 2: Callback Functions for Pie Chart
</H5></div>

**** Add a callback function for `site-dropdown` as input, `success-pie-chart` as output ****

In [9]:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
     Output(component_id = 'success-pie-chart', component_property = 'figure'),
     [Input(component_id = 'site-dropdown', component_property = 'value')]
)
def piegraph_update(site_dropdown):
    if site_dropdown == 'All Sites' or site_dropdown == 'None':
        data  = spacex_launch_data[spacex_launch_data['class'] == 1]
        fig = px.pie(
                data,
                names = 'Launch Site',
                color_discrete_sequence = px.colors.sequential.RdBu ,
                title = '<b>Total Success Launches by Site</b>' ,
            )
        
        #Apply Background Colors to Figure
        fig.update_layout(plot_bgcolor='rgb(74,24,100)')

    else:
        data = spacex_launch_data.loc[spacex_launch_data['Launch Site'] == site_dropdown]
        fig = px.pie(
                data,
                names = 'class',
                color_discrete_sequence = px.colors.qualitative.Set3 ,
                title = '<b>Total Success Launches for Site </b>' + site_dropdown ,
            )
        
        #Apply Background Colors to Figure
        fig.update_layout(plot_bgcolor='rgb(74,24,100)')
    return fig

<div class="alert alert-block alert-warning">
<H5>
TASK 3 & 4: Callback Functions for Scatter Chart with Slider
</H5></div>

**** Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output ****


In [10]:
@app.callback(
     Output(component_id = 'success-payload-scatter-chart', component_property = 'figure'),
     [Input(component_id = 'site-dropdown', component_property = 'value'), 
     Input(component_id = "payload_slider", component_property = "value")]
)

def scattergraph_update(site_dropdown, payload_slider):
    low, high = payload_slider
    if (site_dropdown == 'All Sites' or site_dropdown == 'None'):
        print(payload_slider)
        low, high = payload_slider
        data = spacex_launch_data[spacex_launch_data['Payload Mass (kg)'].between(low, high)]
        fig = px.scatter(
                data, 
                x = "class", 
                y = "Payload Mass (kg)",
                title = '<b>Correlation between Payload and Success for all Sites</b>',
                color = "Booster Version Category"
            )
        #Apply Background Colors to Figure
        fig.update_layout({'plot_bgcolor': colors1b['background'],'paper_bgcolor': colors1b['background'],})
    else:
        print(payload_slider)
        low, high = payload_slider
        data = spacex_launch_data[spacex_launch_data['Payload Mass (kg)'].between(low, high)]
        data_filtered = data[data['Launch Site'] == site_dropdown]
        fig = px.scatter(
                data_filtered,
                x = "class",
                y = "Payload Mass (kg)",
                title = '<b>Correlation between Payload and Success for site </b>'+ site_dropdown,
                color = "Booster Version Category"
            )
        #Apply Background Colors to Figure
        fig.update_layout({'plot_bgcolor': colors1b['background'],'paper_bgcolor': colors1b['background'],})
    return fig

In [11]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "SPACE X DASHBOARD" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
